In [ ]:
# Run this cell in a folder contained by the current project
using Pkg
#Pkg.activate()                 # Walks up the path tree until finds Project.toml
#Pkg.activate(".")              # 
#Pkg.activate("@__DIR__")       # 
Pkg.activate("../")            #
Pkg.instantiate()

In [ ]:
using Plots
using LaTeXStrings
using Graphs
#using MetaGraphs
using MetaGraphsNext
using GraphPlot
using Karnak
using NetworkLayout
using Colors
using Random
using LinearAlgebra
using OffsetArrays
using SparseArrays
using Serialization
using IterativeSolvers
using Krylov
using BenchmarkTools
using Profile
using ProfileView
using Statistics
using DifferentialEquations
using LoopVectorization
using Folds
using Transducers
using Distributed
using JLD2
using OnlineStats
using ImageFiltering
using SavitzkyGolay
using Interpolations
using DataInterpolations
using Polynomials
using CurveFit
using DataFrames
using LsqFit
using ForwardDiff

# Tools

## Enumerator

Tools to compute a running enumeration of a "list" of objects.

In [ ]:
function enum!(d,k)
    return get!(d,k,length(d)+1)
end

### Test

## Running Statistics

Tools to compute running statistics such as mean and std from a stream of numbers.

In [ ]:
mutable struct RunningStat
    n::Int
    m::Real # mean
    s::Real # std
end

function RunningStat()
    return RunningStat(0,0.0,0.0)
end

function Base.push!(rs::RunningStat,v::Real)
    if rs.n>0
        rs.n += 1
        new_m = rs.m + (v-rs.m)/rs.n
        rs.s += (v-rs.m)*(v-new_m)
        rs.m = new_m
    else
        rs.n = 1
        rs.m = v
        rs.s = 0
    end
    return rs.m
end

function Statistics.mean(rs::RunningStat)
    return rs.m 
end

function Statistics.var(rs::RunningStat)
    return rs.s/(rs.n-1)
end

function Statistics.std(rs::RunningStat)
    return sqrt(var(rs))
end

### Tests

## Coefficient of determiantion, $R^2$

Tools to compute the $R^2$ coefficient to evaluate the fit of functions.

In [ ]:
function R_squared(y_data,y_fit)
    y_mean = mean(y_data)
    SS_res = sum((y_data .- y_fit) .^ 2)
    SS_tot = sum((y_data .- y_mean) .^ 2)
    R_sqr = 1 - SS_res / SS_tot
    return R_sqr
end

### Tests

## Sigmoid functions

The logistic sigmoid function is

$$
f(x) = \frac{A}{1+e^{-B(x-x_0)}}
$$

Its derivative is

$$
f'(x) = \frac{A\,B\,e^{-B(x-x_0)}}{(1+e^{-B(x-x_0)})^2}
$$

and its primitive is

$$
F(x) = \int dx\,f(x) = \frac{A}{B}\ln(1+e^{B(x-x_0)})+C
$$


### Test

## Log-range

Tools to create a range of numbers but in logarithmic scale.

In [ ]:
function logrange(start,stepmul,stop)
    @assert start>0
    @assert stepmul>1
    r = []
    s = start
    while s <= stop
        push!(r,s)
        s *= stepmul
    end
    return r
end

### Test

# Generate networks (old stuff)

In [ ]:
function main_component(g)
    c = connected_components(g)
    _,i = findmax(length.(c))
    return g[c[i]]
end

In [ ]:
sorted_connected_components(g) = sort(connected_components(g), by = c -> length(c),rev=true)

## Tests

# Load networks (dict of `(vec_i,vec_j,vec_phi)`)  and save weighted graphs (dict of `(g,phi)`)

# Load weighted graphs (dict of `(g,phi)`)

In [ ]:
@load "../redes/dict_g_phi.jld2" dict_g_phi_jld2

In [ ]:
dict_g_phi_jld2

# Simplicial Complex

Tools to represent simplicial complexes and associated operators in matrix form. For further details, please read de manuscript.

The array `K[k]` is a representation of $K_k$ and has dimensions $m_k\times(k+1)$ and is used to enumerate the $k$-simplices of the complex.
Here $m_k=|K_k|=\dim C_k=\dim C_k^*$ is the number of $k$-simplices in the simplicial complex.
Especifically, `K[k][l,:]` is an array `[i_0,i_1,...,i_k]` denoting the $l$-th $k$-simplex of the complex.
By assumption, $i_0<i_1<...<i_k$.
The dictionary `I[k]` is the inverse mapping of `K[k]`.
It maps arrays `[i_0,i_1,...,i_k]` to their corresponding index `l` in the enumeration.

## Matricial representation of the (co)-boundary and (co)-differential operators

Recall that, if

$$
\partial_k(e_s) = (\alpha_k)_s{}^r e_r
$$
$$
\partial_k^*(e_s) = (\mu_k)_s{}^r e_r 
$$
$$
d_k(e^s) = (\beta_k)^s{}_r e^r
$$
$$
d_k^*(e^s) = (\mu_k)^s{}_r e^r
$$

and $m_{sr}=\delta_{sr}$, then

$$
(\beta_k)^s{}_r = (\alpha_{k+1})_r{}^s
$$
$$
(\mu_k)_s{}^r = (\alpha_{k})_r{}^s
$$
$$
(\nu_k)^s{}_r = (\beta_{k})^r{}_s = (\alpha_{k+1})_s{}^r
$$

and, therefore

$$
(\partial_k(c))^r = (\alpha_k^T)^r{}_sc^s
$$
$$
(\partial_k^*(c))^r = (\alpha_k)^r{}_sc^s
$$
$$
(d_k(f))_r = (\alpha_{k+1})_r{}^sf_s
$$
$$
(d_k^*(f))_r = (\alpha_{k+1}^T)_r{}^sf_s
$$

Finally, note that $\alpha_k\in \mathbb{R}^{m_{k}\times m_{k-1}}$ for $k\in 1,2,...,\kappa$ and

$$
(\alpha_{k})_{i_0...i_j...i_k}{}^{i_0...\hat{i}_j...i_k} = (-1)^j
$$

Here, $d_0$ and $d_1$ are needed. 
Hence, the matrices $\beta_0=\alpha_1$ and $\beta_1=\alpha_2$ have to be computed, respectively. 
In particular, assuming $i<j<k$, it is found that

$$(\alpha_1)_{ij}{}^{j} = 1$$
$$(\alpha_1)_{ij}{}^{i} = -1$$

and

$$(\alpha_2)_{ijk}{}^{jk} = 1$$
$$(\alpha_2)_{ijk}{}^{ik} = -1$$
$$(\alpha_2)_{ijk}{}^{ij} = 1$$

In [ ]:
struct SimplicialComplex
    K::Any
    I::Any
    alpha::Any
end

In [ ]:
"""
Number of dimensions (kappa+1) of a simplicial complex, k = 0,1,...,kappa.
"""
function ndim(sc::SimplicialComplex)
    return length(sc.K)
end

In [ ]:
"""
Maximum dimension of a simplicial complex.
"""
function kappa(sc::SimplicialComplex)
    return ndim(sc)-1
end

In [ ]:
"""
m_k = dim C_k = dim C_k^*.
"""
function m_k(sc::SimplicialComplex,k::Int)
    if k>=0 && k<=kappa(sc)
        return size(sc.K[k])[1]
    end
    return 0
end

In [ ]:
"""
List of the numbers of non-zero elements in the matricial representations of the (dual)-(co)-boundary operators, alpha_k.
"""
function nnz_alpha(sc::SimplicialComplex)
    return [nnz(sc.alpha[k]) for k in 1:length(sc.alpha)] 
end

In [ ]:
"""
Takes as input a list I of dictionaries I_k mapping simplices [i_0,i_1,...,i_k] to their corresponding enumeration numbers l.
Returns a SimplicialComplex containing:
    K : the list of the inverse mappings K_k of the I_k, and
    I : the list of the mappings I_k passed as input,
    alpha : the list alpha of the corresponding matrices alpha_k.
"""
function I_to_SC(I)
    # Create K
    K = []
    for k in 0:(length(I)-1)
        Kk = zeros(Int64,(length(I[k]),k+1))
        for (s,l) in I[k]
            Kk[l,:] = s
        end
        push!(K,Kk)
    end
    K = OffsetArray(K,0:(length(I)-1))

    # Create A
    alpha = []
    for k in 1:(length(I)-1)
        #@show k
        Kk=K[k]
        Kp=K[k-1] # p = k-1
        Ik=I[k]
        Ip=I[k-1]
        n_k = size(Kk)[1]
        n_p = size(Kp)[1]        
        rows = zeros(Int64,(k+1)*n_k)
        cols = zeros(Int64,(k+1)*n_k)
        vals = zeros(Float64,(k+1)*n_k)
        t = 0 # To enumerate the entries of the sparse matrix.
        for l in 1:n_k
            s = Kk[l,:]
            for j in 1:(k+1)
                r = vcat(s[1:(j-1)],s[(j+1):end])
                m = Ip[r]
                t+=1
                rows[t] = l
                cols[t] = m
                vals[t] = 1-2*((j-1)%2)
                #@show s,l,r,m,t,vals[t]
            end
        end
        alpha_k = sparse(rows,cols,vals,n_k,n_p)
        push!(alpha,alpha_k)
    end
    return SimplicialComplex(K,I,alpha)
end

In [ ]:
"""
Takes as input a graph g and returns the list of I_k for k=0,1,2.
"""
function g_to_3_cliques(g)
    # Generate I_0
    I0 = Dict([i] => i for i in 1:nv(g))    
    
    # Generate I_1
    I1 = Dict()    
    for (l,e) in enumerate(edges(g))
        i = src(e)
        j = dst(e)
        I1[[i,j]] = l
    end
    
    # Generate I_2
    I2 = Dict()
    t = 0
    for i in 1:nv(g)
        for j in neighbors(g,i)
            if i<j
                for k in neighbors(g,j)
                    if j<k && has_edge(g,k,i)
                        t += 1
                        I2[[i,j,k]] = t
                    end
                end
            end
        end
    end
    
    I = OffsetArray([I0,I1,I2],0:2)
    return I
end

# Weighted simplicial complex `(sc,phi)` to field `f`

In [ ]:
function phi_ij_to_f(sc,phi_ij)
    #m_0 = m_k(sc,0)
    m_1 = m_k(sc,1)
    f = zeros(m_1)
    K1 = sc.K[1]
    for l in 1:m_1
        i,j = K1[l,:]
        f[l] = phi_ij[j,i]-phi_ij[i,j]
    end
    return f
end

In [ ]:
function phi_i_to_f(sc,phi_ij)
    m_0 = m_k(sc,0)
    m_1 = m_k(sc,1)
    phi_i = zeros(m_0)
    K1 = sc.K[1]
    for l in 1:m_1
        i,j = K1[l,:]
        phi_i[i] += phi_ij[i,j]
        phi_i[j] += phi_ij[j,i]        
    end
    f = zeros(m_1)
    for l in 1:m_1
        i,j = K1[l,:]
        f[l] = phi_i[j]-phi_i[i]
    end
    return f
end

### Tests

#### Testing `g_to_3_cliques` and `I_to_SC` 

#### Testing `phi_to_f`

# Hodge decomposition of $f$

Tools to compute the Hodge decomposition of a 1-chain $f$.

Recall that any $f\in C_1^*$ admits an unique decomposition $f=s+h+g$, where there exists $w\in C_0^*$ such that 

$$
L_0(w)=L^{\uparrow}_0(w)=d_0^*(d_0(w))=d_0^*(g)=d_0^*(f)
$$

and there exists $u\in C_2^*$ such that

$$
L_2^{\downarrow}(u)=d_1(d_1^*(u))=d_1(s)=d_1(f)
$$

In this way, we can find $w$ by solving $L^{\uparrow}_0(w)=d_0^*(f)$ and $u$ by solving $L_2^{\downarrow}(u)=d_1(f)$. Then, we can find $g=d_0(w)$, $s=d_1^*(u)$ and $h=f-g-s$.

## Matricial form

The component-wise form of equation $d_0^*(d_0(w))=d_0^*(f)$ is

$$
(\alpha_1^T)_r{}^s(\alpha_1)_s{}^qw_q = (\alpha_1^T)_r{}^sf_s
$$

or, matricially

$$
\alpha_1^T \alpha_1 w = \alpha_1^T f \;\;\;\; (1)
$$

Similarly, the component-wise form of equation $g = d_0(w)$ is

$$
(\alpha_1)_r{}^s w_s
$$

or, matricially

$$
g = \alpha_1 w
$$

Similarly, the matricial form of equation $d_1(d_1^*(u))=d_1(f)$ is

$$
\alpha_2 \alpha_2^T u = \alpha_2 f \;\;\;\; (2)
$$

and, the matricial form of equation $s=d_1^*(u)$ is

$$
s = \alpha_2^T u
$$

### Test example of Eqs. (18) and (20) in [2].

Eq. (18) of [2] is the analogous of Eq. (1) where $w$ in Eq. (1) is $\alpha$ in Eq. (18) of [2] and $f$ in Eq. (1) is the analogous of $w$ in Eq. (18) of [2].
The matrix of Eq. (18) in [2] is analogous to our $L_0=\alpha_1^T\alpha_1$.
Similarly, Eq. (20) of [2] is the analogous of Eq. (2) where $u$ in Eq. (2) is the analogous of $\gamma$ in Eq. (20) of [2] and $f$ in Eq. (2) is the analogous of $w$ in Eq. (20) of [2].
The matrix of Eq. (20) in [2] is analogous to our $L_2^{\downarrow}=\alpha_2\alpha_2^T$.

The $k$-clique complex of $g$ is the simplicial complex obtained by considering all the $q$-cliques of $g$ for $q\leq k$.

First, the 3-clique complex of the network of Fig. (1) in [2] is created.

Here, $z:=d_0^*(f)$ should equal the r.h.s. of Eq. (18) in [2].

Here, $L_0$ should equal the matrix at the l.h.s. of Eq. (18) in [2].

Next vector should be equal to the vector $\alpha$ reported in Table 5 of [2].

Next vector should equal the r.h.s. of Eq. (18) in [2].

Check that vector $g=d_0(w)=\alpha_1 w$ equals the vector $w_g$ of Table 4 in [2].

Check equality up to numerical tolerance that $d_1(f)=d_1(g)$.

Check the form of $L_2^{\downarrow}=\alpha_2\alpha_2^T$. It should equal the matrix at the l.h.s. of Eq. (20) in [2].

Here, $y:=d_1(f)$ should equal the r.h.s. of Eq. (20) in [2].

The vector $u$ obtained is different from the vector $\gamma$ reported in Table (5) of [2], which is the following one:

Is the vector $u_{\mathrm{johnson}}$ (i.e. $\gamma$) reported in [2] also a solution of Eq. (20) of [2]? We check:

It actually is! What about our solution? Let us check:

It is as well! Hence, there are multiple solutions, so $L_2^{\downarrow}(u)=y$ is a degenerate (i.e. underdetermined) linear system. 

Only one of the solutions of degenerate linear systems of a positive semidefinite linear operator has minimum norm $|x|$. Therefore, we ask ourselves which of the solutions to $L_2^{\downarrow}u=y$ we have found, has the smallest norm? Let us see:

So our solution has smaller norm than that of [2]. Hence, it is a better candidate for the unique solution of minimum norm.

Next, compute $s=d_1^*(u)=\alpha_2^T u$.

Check the "equality" of $\alpha_2 s = d_1(s) = d_1(f) = \alpha_2 f$.

Compute $h$.

Finally check that $L_1(h)=0$.

## Compute Hodge decomposition

The linear equations that should be solved are numerically challenging. Hence, we use the **MINRES-QLP** algoritm, which is specifically devised to solve least square problems of singular or nearly singular linear equations  [9,10,11,12].

In [ ]:
struct HodgeDecomposition
    f::Array{Real}
    s::Array{Real}
    h::Array{Real}
    g::Array{Real}
    u::Array{Real}
    w::Array{Real}    
    k::Int    
end

In [ ]:
function compute_k_th_solenoid_hodge_decomposition(sc,f,k)
    if k<kappa(sc)
        alpha = sc.alpha[k+1]
        y = alpha*f
        L = alpha*alpha'
        u = Krylov.minres_qlp(L,y)[1]
        #u = Krylov.lsqr(L,y)[1]
        s = alpha'*u
        return s,u
    else
        s = zeros(m_k(sc,k)) # s = 0 as a vector enables f=s+h+g to be computed without dimensional problems.
        u = zeros(m_k(sc,k+1))        
        return s,u
    end
end

In [ ]:
function compute_k_th_gradient_hodge_decomposition(sc,f,k)
    if k>=1
        alpha = sc.alpha[k]
        z = alpha'*f
        L = alpha'*alpha
        w = Krylov.minres_qlp(L,z)[1]
        #w = Krylov.lsqr(L,z)[1]
        g = alpha*w
        return g,w
    else
        g = zeros(m_k(sc,k)) # g = 0 as a vector enables f=s+h+g to be computed without dimensional problems.
        w = zeros(m_k(sc,k-1))        
        return g,w
    end
end

In [ ]:
function compute_k_th_hodge_decomposition(sc,f,k)
    @assert length(f) == m_k(sc,k)
    s,u = compute_k_th_solenoid_hodge_decomposition(sc,f,k)
    g,w = compute_k_th_gradient_hodge_decomposition(sc,f,k)
    h=f-s-g
    return HodgeDecomposition(f,s,h,g,u,w,k)
end

### Test

#### Basic test

Next, we create a 1-chain $f$ of components $f_{ij}=w_i-w_j$ where $w$ is the 0-chain of components $w_i=i$.

#### Test with futbol networks

# Ranking tools

Tools to generate **true** ratings and rankings for a given network.

In [ ]:
"""
Takes a vector and shifts the values of its components by a constant to make all of them non-negative.
"""
function ratings(w)
    return w .- minimum(w)
end

In [ ]:
"""
Normalizes a vector of ratings to have values between 0 and 1 inclusive.
"""
function normalized_ratings(w)
    minw = minimum(w)
    #maxw = maximum(w)
    #return (w .- minw)/(maxw-minw)
    return (w .- minw)/length(w)
end

In [ ]:
"""
Takes a vector of ratings and computes the corresponding ranking
"""
function ranking(w)
    return sortperm(w,rev=true)
end

In [ ]:
"""
Given two vectors v and u, this computes the absolute differences

    x_i = |v_i-u_i|

And returns the vector w of cumulative diferences

    w_i = sum_{j <= i} x_j

This can be used to compare two vectors of ratings or rankings
"""
function compute_cumulative_abs_diff(v,u)
    n = length(v)
    @assert length(u) == n
    cumulative_abs_diff = zeros(n)
    sum = 0
    for i in 1:n
         sum += abs.(v[i]-u[i])
         cumulative_abs_diff[i] = sum
    end
    return cumulative_abs_diff
end

### Test

# Inference of ratings and rankings

General idea:

* There are 5 soccer leagues: England, France, Germany, Italy and Spain.
* Each has 20 teams, with the exception of Germany which has 18.
* Each pair of teams of each league plays two games, one as local and another as visitor.
* For each pair of games between two teams $i$ and $j$ of a given league, one as local and one as visitor, a score $\phi_{ij}\in \mathbb{R}$ is provided.
* Each score matrix $\phi$ defines a non-directed fully connected network.
* A fraction $p\in [0,1]$ of the links of the networks can be removed using some criterium:
    * uniformly at random, or
    * in increasing order of absolute weight $|\phi_{ij}|$.
* The networks are promoted to simplicial complexes where:
    * nodes $i$ become 0-simplices,
    * links $ij$ become 1-simplices, and
    * connected triplets $ijk$ become 2-simplices, where $ijk$ is a connected triplet of a network if $ij$, $jk$ and $ik$ are links of the network.
* A 1-cochain of coordinates $f_{ij}=\phi_{ji}-\phi_{ij}$ is defined over each simplicial complex.
* Alternatively, a 1-cochain of coordinates $f_{ij}=\phi_{j}-\phi_{i}$ where $\phi_i = \sum_j \phi_{ij}$ is the strength of node $i$, is defined over each simplicial complex.

### Color schemes

In [ ]:
palette_colors = palette(:auto)
col1 = palette_colors[1]
col2 = palette_colors[2]
col3 = palette_colors[3]
col4 = palette_colors[4]
col5 = palette_colors[5]
col6 = palette_colors[6]
col7 = palette_colors[7]
col8 = palette_colors[8]
cols = [col1,col2,col3,col4,col5,col6,col7,col8]

In [ ]:
#colors_N = palette(:roma,6)
#colors_N = palette(:thermal,8)
#colors_N = palette(:haline,6)
cols1 = [col1,col2]
colors_N = palette(cols1,6)

In [ ]:
#colors_N = palette(:roma,6)
#colors_N = palette(:thermal,8)
#colors_N = palette(:haline,8)
#colors_z = palette(:hawaii,13)
cols2 = [col4,col6]
colors_z = palette(cols2,13)
colors_k = colors_z
colors_q = colors_z
colors_p = colors_z

## Experiment 1:

For each league, compute the ratings $w_i$ and rankings $r_i$ for:

* fully connected networks, and
* 1-cochain given by $f_{ij}=\phi_{ji}-\phi_{ij}$.

In [ ]:
dict_sc_hd_E1_jld2 = Dict()
for (country,score) in keys(dict_g_phi_jld2)
    #@show (country,score)
    (g,phi_ij) = dict_g_phi_jld2[(country,score)]
    I_3c = g_to_3_cliques(g)
    sc = I_to_SC(I_3c)
    f = phi_ij_to_f(sc,phi_ij)
    hd = compute_k_th_hodge_decomposition(sc,f,1)
    dict_sc_hd_E1_jld2[(country,score)] = (sc,hd)
end

In [ ]:
@save "../redes/dict_sc_hd_E1.jld2" dict_sc_hd_E1_jld2

In [ ]:
@load "../redes/dict_sc_hd_E1.jld2" dict_sc_hd_E1_jld2

In [ ]:
dict_sc_hd_E1_jld2

In [ ]:
countries = Dict()
scores = Dict()
for (country,score) in keys(dict_sc_hd_E1_jld2)
    enum!(countries,country)
    enum!(scores,score)
end
scores

In [ ]:
improved_score_titles = Dict()
improved_score_titles["counterattack"] = "counterattacks"
improved_score_titles["crossing"] = "crossings"
improved_score_titles["direct_play"] = "direct play"
improved_score_titles["T_maintenance"] = "maintenance time"
improved_score_titles["flow_rate"] = "flow rate"
improved_score_titles["T_build_up"] = "build-up time"
improved_score_titles["shots"] = "shots"
improved_score_titles["T_zona_media"] = "middle-zone time"
improved_score_titles["pressure_point"] = "pressure points"
improved_score_titles["pressure_loss"] = "pressure losses"

In [ ]:
for (score,cs) in scores
    p=plot(title=improved_score_titles[score])
    plot!(xlabel=L"r")
    plot!(ylabel=L"w_r")
    #plot!(xscale=:log10)
    #plot!(yscale=:log10)
    for (country,cc) in countries    
        (sc,hd) = dict_sc_hd_E1_jld2[(country,score)]
        wmin = minimum(hd.w)
        wmax = maximum(hd.w)
        wdelta = wmax-wmin
        yvals = hd.w
        #yvals = hd.w .- wmin .+ wdelta/length(hd.w)
        plot!(yvals,label="",c=cc)
        scatter!(yvals,label=country,c=cc)
    end
    savefig(p,"plots/E1/rating-vs-original-ranking-"*score*".pdf")
end

In [ ]:
for (score,cs) in scores
    p=plot(title=improved_score_titles[score])
    plot!(xlabel=L"r")
    plot!(ylabel=L"\hat{r}")
    plot!(xlims=(0,20.5))
    plot!(ylims=(0,20.5))
    #plot!(xscale=:log10)
    #plot!(yscale=:log10)
    for (country,cc) in countries    
        (sc,hd) = dict_sc_hd_E1_jld2[(country,score)]
        #wmin = minimum(hd.w)
        #wmax = maximum(hd.w)
        #wdelta = wmax-wmin
        #yvals = hd.w
        #yvals = hd.w .- wmin .+ wdelta/length(hd.w)
        yvals = ranking(hd.w)
        plot!(yvals,label="",c=cc)
        scatter!(yvals,label=country,c=cc)
    end
    savefig(p,"plots/E1/inferred-ranking-vs-original-ranking-"*score*".pdf")
end

## Experiment 2:

For each league, compute the ratings $w_i$ and rankings $r_i$ for:

* fully connected networks, and
* 1-cochain given by $f_{ij}=\phi_{j}-\phi_{i}$ with $\phi_i = \sum_j \phi_{ij}$.

In [ ]:
dict_sc_hd_E2_jld2 = Dict()
for (country,score) in keys(dict_g_phi_jld2)
    #@show (country,score)
    (g,phi_ij) = dict_g_phi_jld2[(country,score)]
    I_3c = g_to_3_cliques(g)
    sc = I_to_SC(I_3c)
    f = phi_i_to_f(sc,phi_ij)
    hd = compute_k_th_hodge_decomposition(sc,f,1)
    dict_sc_hd_E2_jld2[(country,score)] = (sc,hd)
end

In [ ]:
@save "../redes/dict_sc_hd_E2.jld2" dict_sc_hd_E2_jld2

In [ ]:
@load "../redes/dict_sc_hd_E2.jld2" dict_sc_hd_E2_jld2

In [ ]:
dict_sc_hd_E2_jld2

In [ ]:
for (score,cs) in scores
    p=plot(title=improved_score_titles[score])
    plot!(xlabel=L"r")
    plot!(ylabel=L"w_r")
    #plot!(xscale=:log10)
    #plot!(yscale=:log10)
    for (country,cc) in countries    
        (sc,hd) = dict_sc_hd_E2_jld2[(country,score)]
        wmin = minimum(hd.w)
        wmax = maximum(hd.w)
        wdelta = wmax-wmin
        yvals = hd.w
        #yvals = hd.w .- wmin .+ wdelta/length(hd.w)
        plot!(yvals,label="",c=cc)
        scatter!(yvals,label=country,c=cc)
    end
    savefig(p,"plots/E2/rating-vs-original-ranking-"*score*".pdf")
end

In [ ]:
for (score,cs) in scores
    p=plot(title=improved_score_titles[score])
    plot!(xlabel=L"r")
    plot!(ylabel=L"\hat{r}")
    plot!(xlims=(0,20.5))
    plot!(ylims=(0,20.5))
    #plot!(xscale=:log10)
    #plot!(yscale=:log10)
    for (country,cc) in countries    
        (sc,hd) = dict_sc_hd_E2_jld2[(country,score)]
        #wmin = minimum(hd.w)
        #wmax = maximum(hd.w)
        #wdelta = wmax-wmin
        #yvals = hd.w
        #yvals = hd.w .- wmin .+ wdelta/length(hd.w)
        yvals = ranking(hd.w)
        plot!(yvals,label="",c=cc)
        scatter!(yvals,label=country,c=cc)
    end
    savefig(p,"plots/E2/inferred-ranking-vs-original-ranking-"*score*".pdf")
end

In [ ]:
N = 10000
mean_k = 3.0
p = mean_k/(N-1)
@show N
@show mean_k
@show p
#g = erdos_renyi(6, 0.4)
g = erdos_renyi(N,p)
@show nv(g)
@show ne(g)
@show 2*ne(g)/N
#@show g
;

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
w = gen_artificial_ratings(1:m_k(sc,0))

In [ ]:
f = gen_artificial_f(sc,w)

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,f,1)

In [ ]:
hd.w

In [ ]:
ratings(hd.w)

In [ ]:
ranking(hd.w)

In [ ]:
cumdiff_w = compute_cumulative_abs_diff(normalized_ratings(w),normalized_ratings(hd.w))

In [ ]:
N = length(cumdiff_w)
n = N-1
plot(title="ER")
plot!(xlabel=L"i/n")
plot!(ylabel=L"\rho_w(i)") # hat{w} is the inferred rating
plot!((0:n)/n,cumdiff_w/N,label="")

In [ ]:
plot(title="ER")
plot!(xlabel=L"\hat{r}_i")
plot!(ylabel=L"r_i")
scatter!(ranking(w),ranking(hd.w),label="")

In [ ]:
sum(abs.(ranking(w)-ranking(hd.w)))

In [ ]:
I_3c = g_to_3_cliques(g)

In [ ]:
sc = I_to_SC(I_3c)

In [ ]:
w = gen_artificial_ratings(1:m_k(sc,0))

In [ ]:
f = gen_artificial_f(sc,w)

In [ ]:
hd = compute_k_th_hodge_decomposition(sc,f,1)

In [ ]:
hd.w

In [ ]:
ratings(hd.w)

In [ ]:
ranking(hd.w)

In [ ]:
cumdiff_w = compute_cumulative_abs_diff(normalized_ratings(w),normalized_ratings(hd.w))

In [ ]:
N = length(cumdiff_w)
n = N-1
plot(title="BA")
plot!(xlabel=L"i/n")
plot!(ylabel=L"\rho_w(i)") # hat{w} is the inferred rating
plot!((0:n)/n,cumdiff_w/N,label="")

In [ ]:
plot(title="BA")
plot!(xlabel=L"\hat{r}_i")
plot!(ylabel=L"r_i")
scatter!(ranking(w),ranking(hd.w),label="")

In [ ]:
list_cc_mean[1]/list_cc_mean[1]